# Recommending movies: retrieval

Real world Recommender systems are often made up of following steps - 

* `Retrieval` :   Selecting few thousands of possible candidates to recommend from a set of millions. Because the retrieval model may be dealing with millions of candidates, it has to be computationally efficient.

* `Ranking` : Drill down thousand possible candidates to few hundreds

* `Post-Ranking` : Further refine the candidates to few dozens - Might be helpful in case user is logged from mobile device where screen real estate is limited

In this notebook, we will go over Retrieval stage of the Recommender System. Retrieval models are often composed of two sub-models:

* A `query model` computing the query representation (normally a fixed-dimensionality embedding vector) using query features.
* A `candidate model` computing the candidate representation (an equally-sized vector) using the candidate features

The outputs of the two models are then multiplied together to give a query-candidate affinity score, with higher scores expressing a better match between the candidate and the query.

## Dataset

The Movielens dataset is a classic dataset from the GroupLens research group at the University of Minnesota. It contains a set of ratings given to movies by a set of users, and is a workhorse of recommender system research.

The data can be treated in two ways:

1. It can be interpreted as expressesing which movies the users watched (and rated), and which they did not. This is a form of `implicit feedback`, where users' watches tell us which things they prefer to see and which they'd rather not see.

2. It can also be seen as expressesing how much the users liked the movies they did watch. This is a form of `explicit feedback`: given that a user watched a movie, we can tell roughly how much they liked by looking at the rating they have given.


In this tutorial, we are focusing on a retrieval system: a model that predicts a set of movies from the catalogue that the user is likely to watch. Often, implicit data is more useful here, and so we are going to treat Movielens as an implicit system. This means that every movie a user watched is a positive example, and every movie they have not seen is an implicit negative example.

## Imports

In [1]:
! pip install -q tensorflow-recommenders
! pip install -q --upgrade tensorflow-datasets
! pip install -q scann
! pip install tfds-nightly

In [2]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

## EDA

In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [4]:
# Let's look at features from ratings datasets
for x in ratings.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [5]:
# And features from movies datasets
for x in movies.take(1).as_numpy_iterator():
  pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [6]:
# Extract relevant features for our task - User ID and Movie Title
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

### Train-Test Split

We'll go for 80-20 split

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [8]:
# Let's also figure out unique user ids and movie titles present in the data.

# This is important because we need to be able to map the raw values of our categorical features to embedding vectors in our models. 
# To do that, we need a vocabulary that maps a raw feature value to an integer in a contiguous range: 
# this allows us to look up the corresponding embeddings in our embedding tables.

In [9]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_movie_titles[:10]

array([b"'Til There Was You (1997)", b'1-900 (1994)',
       b'101 Dalmatians (1996)', b'12 Angry Men (1957)', b'187 (1997)',
       b'2 Days in the Valley (1996)',
       b'20,000 Leagues Under the Sea (1954)',
       b'2001: A Space Odyssey (1968)',
       b'3 Ninjas: High Noon At Mega Mountain (1998)',
       b'39 Steps, The (1935)'], dtype=object)

## Model Implementation

Because we are building a two-tower retrieval model, we can build each tower separately and then combine them in the final model.

### The query tower

In [10]:
# The first step is to decide on the dimensionality of the query and candidate representations:
embedding_dimension = 32

# Higher values will correspond to models that may be more accurate, 
# but will also be slower to fit and more prone to overfitting.

In [11]:
# The second is to define the model itself. 
# Here, we're going to use Keras preprocessing layers to first convert user ids to integers,
# and then convert those to user embeddings via an Embedding layer. 
# Note that we use the list of unique user ids we computed earlier as a vocabulary

user_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_user_ids, mask_token=None),
  # We add an additional embedding to account for unknown tokens.
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### The candidate tower

In [12]:
# Same approach as the query tower

movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### Metrics

The input to the model is (user-movie) pair. This will be treated as a positive pair. If our model predicts higher score for the input (user-movie) pair than any other (user-movie) pair, then our models is performing great!

To achieve this, we use FactorizedTopK as the metric, which takes as input - candidates.

In our case, we will pass movies to compare affinity score wrt the input. These movies need to be mapped to embeddings and we'll the movie model created above

In [13]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

### Loss

TFRS has several loss layers and tasks to make this easy.

In this instance, we'll make use of the Retrieval task object: a convenience wrapper that bundles together the loss function and metric computation

In [14]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

The task itself is a Keras layer that takes the query and candidate embeddings as arguments, 
and returns the computed loss: we'll use that to implement the model's training loop.

### Full Model

We can now put it all together into a model. TFRS exposes a base model class (tfrs.models.Model) which streamlines building models: all we need to do is to set up the components in the `__init__` method, and implement the compute_loss method, taking in the raw features and returning a loss value.

The base model will then take care of creating the appropriate training loop to fit our model.

In [15]:
class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # We pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # And pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # The task computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

The tfrs.Model base class is a simply convenience class: 

it allows us to compute both training and test losses using the same method.

### Fitting and evaluating

After defining the model, we can use standard Keras fitting and evaluation routines to fit and evaluate the model.

Let's first instantiate the model.

In [16]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [17]:
# Then shuffle, batch, and cache the training and evaluation data.

cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [18]:
# Train the model
model.fit(cached_train, epochs=10)

Epoch 1/10
10/10 [==============================] - 29s 2s/step - factorized_top_k/top_1_categorical_accuracy: 2.8750e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0042 - factorized_top_k/top_10_categorical_accuracy: 0.0111 - factorized_top_k/top_50_categorical_accuracy: 0.0794 - factorized_top_k/top_100_categorical_accuracy: 0.1548 - loss: 69885.1129 - regularization_loss: 0.0000e+00 - total_loss: 69885.1129
Epoch 2/10
10/10 [==============================] - 24s 2s/step - factorized_top_k/top_1_categorical_accuracy: 0.0021 - factorized_top_k/top_5_categorical_accuracy: 0.0170 - factorized_top_k/top_10_categorical_accuracy: 0.0346 - factorized_top_k/top_50_categorical_accuracy: 0.1627 - factorized_top_k/top_100_categorical_accuracy: 0.2866 - loss: 67523.3672 - regularization_loss: 0.0000e+00 - total_loss: 67523.3672
Epoch 3/10
10/10 [==============================] - 27s 3s/step - factorized_top_k/top_1_categorical_accuracy: 0.0029 - factorized_top_k/top_5_categorical_accuracy:

`IMPORTANT`

As the model trains, the loss is falling and a set of top-k retrieval metrics is updated. 
These tell us whether the true positive is in the top-k retrieved items from the entire candidate set. 

> For example, a top-5 categorical accuracy metric of 0.2 would tell us that, on average, the true positive is in the top 5 retrieved items 20% of the time.

Note that, in this example, we evaluate the metrics during training as well as evaluation. Because this can be quite slow with large candidate sets, it may be prudent to turn metric calculation off in training, and only run it in evaluation.

In [19]:
# Finally, we can evaluate our model on the test set:

model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 8s 984ms/step - factorized_top_k/top_1_categorical_accuracy: 2.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0033 - factorized_top_k/top_10_categorical_accuracy: 0.0100 - factorized_top_k/top_50_categorical_accuracy: 0.0839 - factorized_top_k/top_100_categorical_accuracy: 0.1819 - loss: 31678.6315 - regularization_loss: 0.0000e+00 - total_loss: 31678.6315


{'factorized_top_k/top_100_categorical_accuracy': 0.18185000121593475,
 'factorized_top_k/top_10_categorical_accuracy': 0.010049999691545963,
 'factorized_top_k/top_1_categorical_accuracy': 0.0002500000118743628,
 'factorized_top_k/top_50_categorical_accuracy': 0.08389999717473984,
 'factorized_top_k/top_5_categorical_accuracy': 0.0032999999821186066,
 'loss': 28730.44140625,
 'regularization_loss': 0,
 'total_loss': 28730.44140625}

Test set performance is much worse than training performance. This is due to two factors:

Our model is likely to perform better on the data that it has seen, simply because it can memorize it. This overfitting phenomenon is especially strong when models have many parameters. It can be mediated by model regularization and use of user and movie features that help the model generalize better to unseen data.
The model is re-recommending some of users' already watched movies. These known-positive watches can crowd out test movies out of top K recommendations.
The second phenomenon can be tackled by excluding previously seen movies from test recommendations. This approach is relatively common in the recommender systems literature, but we don't follow it in these tutorials. If not recommending past watches is important, we should expect appropriately specified models to learn this behaviour automatically from past user history and contextual information. Additionally, it is often appropriate to recommend the same item multiple times (say, an evergreen TV series or a regularly purchased item).

## Making predictions

Now that we have a model, we would like to be able to make predictions. We can use 2 ways for predictions - 

1. Brute Force way -  We can use the tfrs.layers.factorized_top_k.BruteForce layer to do this. The brute force layers retreives the output using an exhaustive search. Kinda slow method, but works!

2. Approximate retrieval index using ScanN - We can also export an approximate retrieval index to speed up predictions. This will make it possible to efficiently surface recommendations from sets of tens of millions of candidates.This layer will perform approximate lookups: this makes retrieval slightly less accurate, but orders of magnitude faster on large candidate sets.

### Brute Force Method

In [20]:
# Create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)' b'Rudy (1993)'
 b'Aristocats, The (1970)']


### Approximate Lookup using ScanN

we can use the scann package. This is an optional dependency of TFRS, and we installed it separately at the beginning of this tutorial by calling !pip install -q scann.

In [21]:
scann_index = tfrs.layers.factorized_top_k.ScaNN(model.user_model)
scann_index.index_from_dataset(
  tf.data.Dataset.zip((movies.batch(100), movies.batch(100).map(model.movie_model)))
)

# Get recommendations.
_, titles = scann_index(tf.constant(["42"]))
print(f"Recommendations for user 42: {titles[0, :3]}")

Recommendations for user 42: [b'Bridges of Madison County, The (1995)' b'Affair to Remember, An (1957)'
 b'Rent-a-Kid (1995)']
